In [1]:
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from nltk.tokenize import word_tokenize

import numpy as np
import json
from bs4 import BeautifulSoup
import torch
import gzip

from transformers import DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoderTokenizer
from transformers import DPRQuestionEncoder
from transformers import DPRContextEncoder
import csv
from transformers import BertModel, BertTokenizer, BertTokenizerFast
from torch.nn import CosineSimilarity
from torch.utils.data import DataLoader, Dataset
from nltk import word_tokenize
import pandas as pd
import random

from IPython import embed
from sklearn.metrics import classification_report
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
device = "cpu"
torch.cuda.empty_cache()

In [2]:
class SimpleDataset(Dataset):
    def __init__(self, context_file, question_file, idxs, NQ, map):
        context_vals = torch.load(context_file) 
        question_vals = torch.load(question_file)
        self.context_embeds = []
        for elem in context_vals:
            self.context_embeds.append(torch.reshape(elem, (1, 768)))
        self.query_embeds = []
        for elem in question_vals:
            self.query_embeds.append(torch.reshape(elem, (1, 768)))
        self.ixs = idxs
        if not NQ:
            self.map = map
        else:
            self.map = {}
            for i in range(len(idxs)):
                self.map[str(i)] = i

    def __len__(self):
        return len(self.ixs)

    def __getitem__(self, idx):
        # query embed, context embeds, true label
        batch_ixs = self.ixs[idx]
        query_embed = self.query_embeds[idx]
        context_embeds = [self.context_embeds[int(batch_ixs[i])]  for i in range(len(batch_ixs))  ]
        label = batch_ixs[0]
        return ( query_embed, context_embeds, batch_ixs[:], label )

        


In [3]:
cos = CosineSimilarity(dim=0, eps=1e-6)
def run_DPR(query_embed, context_embeds, ixs):
    similarities = []
    for i in range(len(context_embeds)):
        similarities.append( torch.dot(query_embed[0], context_embeds[i][0])/torch.norm(context_embeds[i][0]) )
        #embed()
        #a = normalize(e)
        #b = normalize(query_embed)
        
        #similarities.append( CosineSimilarity(dim=-1, eps=1e-6)(e, query_embed)  )
    return int(ixs[np.argmax(np.array(similarities))])   #, dim = 0)

In [4]:
def NQ_processing(idx_f):
    
    rows = []
    with open(idx_f, newline='') as idx_file:
        for line in csv.reader(idx_file):
            rows.append(list(line))
    idx_file.close()
    return rows
    


In [15]:
#For Dev Datasets
i = 0
question_file = f"/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/dev/question-{i}-embeds"
context_file = f"/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/dev/context-{i}-embeds"
idx_file = f"/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/dev/nq-dev-ix-{i}.csv"

In [18]:
#For Training Datasets
i = 0
question_file = f"/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/train/question-{i}-embeds"
context_file = f"/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/train/context-{i}-embeds"
idx_file = f"/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/train/nq-train-ix-{i}.csv"

In [5]:
#For Squad
question_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/squad/dev-questions"
context_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/squad/dev-contexts-0"
idx_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/squad/dev_ixs.csv"

In [5]:
#FOR PUBMED
question_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/pubmed/dev-questions"
context_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/pubmed/dev-contexts-0"
idx_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/pubmed/dev_ixs.csv"

In [5]:
#FOR SCOTUS
question_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/scotus-flex/dev-questions"
context_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/scotus-flex/dev-contexts-0"
idx_file = "/home/ubuntu/nlm/noah/scotus/dev_ix.csv"

In [19]:
#FOR NFCORPUS
question_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/nfcorpus/dev-questions"
context_file = "/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/nfcorpus/dev-contexts-0"
idx_file = "/home/ubuntu/nlm/noah/nfcorpus/dev-clean-ix.csv"

In [20]:
#DATASET FOR NQ/PUBMED
rows = NQ_processing(idx_file)
dataset = SimpleDataset(context_file, question_file, rows, True, [])
dataloader = DataLoader(dataset, batch_size=50)

In [6]:
#DATASET FOR SQUAD
rows = NQ_processing(idx_file)
file_name = "/home/ubuntu/nlm/noah/scotus/dev.json"
with open(file_name) as file: 
    data = json.load(file)
QC_map = data['map']
dataset = SimpleDataset(context_file, question_file, rows, False, QC_map)
dataloader = DataLoader(dataset, batch_size=50)

In [7]:
y_pred = []
y_label = []
#BASIC DPR (NO DATALOADER)
for i in tqdm(range(len(rows))):
    query, context, ixs, label = dataset[i]
    pred = run_DPR(query, context, ixs)
    # print("PRED:", pred, "LABEL:", label)
    # print(sims)
    y_pred.append(pred)
    y_label.append(int(label)) 
print(classification_report(y_label, y_pred, digits=4))

100%|██████████| 271/271 [00:00<00:00, 1052.10it/s]

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         1
           1     0.0000    0.0000    0.0000         1
           2     0.0000    0.0000    0.0000         1
           3     0.0000    0.0000    0.0000         1
           4     0.5000    1.0000    0.6667         1
           5     0.0000    0.0000    0.0000         1
           6     0.0000    0.0000    0.0000         1
           7     0.0000    0.0000    0.0000         1
           8     0.0000    0.0000    0.0000         1
           9     0.0000    0.0000    0.0000         1
          10     0.0000    0.0000    0.0000         1
          11     0.0000    0.0000    0.0000         1
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         1
          14     0.0000    0.0000    0.0000         1
          15     0.0000    0.0000    0.0000         1
          16     0.0000    0.0000    0.0000         1
          17     0.0000    


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
#BASIC DPR 

y_pred = []
y_label = []
for step, (query, context, ixs, label) in tqdm(enumerate(dataloader)):
    context_embeds = []
    # if step == 2: break
    for j in range(len(context)):
        new = []
        for k in range(len(context[j])):
            new.append(context[j][k])
        context_embeds.append(torch.stack(new))
    context_embeds = torch.stack(context_embeds).permute(1,0,2,3)
    inds = []
    for m in range(len(ixs)):
        new = []
        for n in range(len(ixs[m])):
            new.append(int(ixs[m][n]))
        inds.append(new)
    inds = torch.tensor(inds).permute(1, 0)    
    for i in range(len(query)): # this should be batch size
        #embed()
        question_embed = query[i] #.detach()
        #embed()
        pred = run_DPR(question_embed, context_embeds[i], inds[i])
        y_pred.append(pred)
        y_label.append(int(label[i]))
        break
    break
print(classification_report(y_label, y_pred, digits=4))
    
    


0it [00:00, ?it/s]

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000         1

    accuracy                         1.0000         1
   macro avg     1.0000    1.0000    1.0000         1
weighted avg     1.0000    1.0000    1.0000         1



In [ ]:
a = dataset.context_embeds[0]
print(a)

In [ ]:
s = "Normans The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."
with torch.no_grad():
    context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')#.to(device)
    context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
tokenized = context_tokenizer(s, padding='max_length', max_length = 512,truncation=True)
batch_embeds = context_encoder( torch.tensor([tokenized['input_ids']]))[0]
print(batch_embeds)
vals = torch.load("/home/ubuntu/nlm/williamyang/DPR_Preprocess_Data/squad/dev-contexts-0")
print(vals[0])

In [5]:
#BM25 

def NQ_file_open(file_object, device):
    while True:
        try:
            chunk = json.loads(file_object.readline())
            yield chunk
        except:
            yield "NONE"

def NQ_get_docs(f):
    examples = {'docs': [], 'questions': []}
    count = 0
    cap = 13000 #number of docs per question set in a dataset
    with gzip.open(f, 'r') as file:
        for chunk in NQ_file_open(file, device):
            if count > cap: 
                break
            if chunk == "NONE":
                break
            examples['questions'].append(chunk['question_text'].split(" "))
            doc_string = []
            for elem in chunk['document_tokens']:
                if not elem['html_token']:
                    doc_string.append(elem['token'])
            examples['docs'].append(doc_string)
            count += 1
    file.close()
    return examples
score = 0

examples = NQ_get_docs("/home/ubuntu/nlm/williamyang/Data/NQ/dev/nq-dev-00.jsonl.gz")

    








In [22]:
file_name = "/home/ubuntu/nlm/noah/nfcorpus/dev-clean.json"
with open(file_name) as file: 
    examples = json.load(file)


In [15]:
def NQ_file_open(file_object, device):
    while True:
        try:
            chunk = json.loads(file_object.readline())
            yield chunk
        except:
            yield "NONE"

f = '/home/ubuntu/nlm/williamyang/Data/NQ/train/nq-train-00.jsonl.gz'
examples = {'text': [], 'question': []}
count = 0
cap = 13000 #number of docs per question set in a dataset
with gzip.open(f, 'r') as file:
    for chunk in NQ_file_open(file, device):
        if count > cap: 
            break
        if chunk == "NONE":
            break
        examples['question'].append(chunk['question_text'].split(" "))
        doc_string = []
        for elem in chunk['document_tokens']:
            if not elem['html_token']:
                doc_string.append(elem['token'])
        examples['text'].append(doc_string)
        count += 1
file.close()


In [24]:
#bm25 = BM25Okapi([word_tokenize(a) for a in examples['text']])
bm25 = BM25Okapi([word_tokenize(a) for a in examples['text']])
print("STARTING SCORING")
y_pred = []
y_label = []
for i in tqdm(range(len(examples['question']))):
    tokenized_query = examples['question'][i]
    doc_scores = bm25.get_scores(word_tokenize(tokenized_query))
    y_pred.append(np.argmax(doc_scores))
    y_label.append(examples['map'][str(i)])
    #y_label.append(i)
    # if label == pred: score += 1
    # print("PRED: ", pred)
    # print("LABEL: ", label)
print(classification_report(y_label, y_pred, digits=4))

STARTING SCORING


100%|██████████| 102/102 [00:00<00:00, 2534.67it/s]

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         1
           1     0.0000    0.0000    0.0000         1
           2     0.0000    0.0000    0.0000         1
           3     0.0000    0.0000    0.0000         1
           4     0.0000    0.0000    0.0000         1
           5     0.0000    0.0000    0.0000         1
           6     0.0000    0.0000    0.0000         1
           7     0.0000    0.0000    0.0000         1
           8     0.0000    0.0000    0.0000         1
           9     0.0000    0.0000    0.0000         1
          10     0.0000    0.0000    0.0000         1
          11     0.5000    1.0000    0.6667         1
          12     0.0000    0.0000    0.0000         1
          13     0.0000    0.0000    0.0000         1
          14     0.0000    0.0000    0.0000         1
          15     0.0000    0.0000    0.0000         1
          16     0.0000    0.0000    0.0000         1
          17     1.0000    


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
print(examples['question'][0])

['what', 'purpose', 'did', 'seasonal', 'monsoon', 'winds', 'have', 'on', 'trade']
